In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
%load_ext autoreload
%autoreload 2

from matplotlib.ticker import FuncFormatter

import matplotlib.ticker as mtick
import math  

from scipy import stats
import statsmodels.stats.proportion as smprob
import statsmodels.stats.contingency_tables
import statsmodels.api as sm
import datetime
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import sys
import statistics
from scipy.stats import sem
from matplotlib import colors as mcolors


colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)
from statistics import geometric_mean
from scipy.stats import gstd

from scipy import stats

import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

from statistics import mean
import seaborn as sns

from scipy import stats
import statistics

pd.options.mode.chained_assignment = None  # default='warn'

from scipy.stats import mannwhitneyu

In [ ]:
statsmodels.__version__

## Second dose and booster

In [ ]:
booster_first_orig_pre = pd.read_csv('c:/temp/vaccine_antibody/out_for_analysis_compare_first_25112021.txt', delimiter='\t')

In [ ]:
booster_first_orig = booster_first_orig_pre[booster_first_orig_pre['first_val']>=50]

In [ ]:
booster_first_orig.shape

In [ ]:
booster_first_orig.columns

In [ ]:
analysis_1_ids = booster_first_orig['id'].tolist()

In [ ]:
len(analysis_1_ids)

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(14,5),gridspec_kw={'width_ratios': [1,1]})
colors = sns.color_palette(n_colors=6)
my_width = 3

my_linestyle = 'solid'
gap = 0.1
my_label = 'Observed GM'
my_label_dots =  'Observed'
vec_x_ticks_0=  [0,30,60,65,90,120]
vec_x_ticks_l_0=  [0,30,60,0,30,60]

vec_x_ticks_1=  [0,30,60,65,90,120]
vec_x_ticks_l_1=  [0,30,60,0,30,60]

# vec_x_ticks=  [20,30,60,90]
# vec_x_ticks_l=  ['',30,60,90]

vec_diff_control_vac2 = []
vec_diff_control_booster = []
vec_diff_case_vac2 = []
vec_diff_case_booster = []


vec_vals_control_vac2 = []
vec_vals_control_booster = []
vec_vals_case_vac2 = []
vec_vals_case_booster = []

vec_fold_control = []
vec_fold_case = []

vec_fold_control_orig = []
vec_fold_case_orig = []

vec_age_control =[]
vec_age_case =[]


for temp_booster in [0,1] : 
    if (temp_booster==0) : print('@@@@@  control @@@@@@')
    else :  print('@@@@@  case @@@@@@')
    
    
    j = temp_booster
    vals = []
    lower = []
    upper = []
    vec_x = []
    ax_temp = ax[temp_booster]
    
    dots_x = []
    dots_y = []
    
    vec_first = []
    vec_booster = []
    
    vec_first_orig = []
    vec_booster_orig = []    

    for temp_group in ['first_val','booster_val']:
            
        if (temp_group=='first_val'):
            log_col = 'log_first'
            real_col = 'first_val'
            diff_col = 'diff_between_val1_and_sec_vacc'
            x_val =1
            x_lag = 0
        else:
            log_col = 'log_booster'
            real_col = 'booster_val'
            diff_col = 'diff_booster'
            x_val = 3
            x_lag = 65

        diff_days = booster_first_orig[(booster_first_orig['diff_booster']>=0) & (booster_first_orig['case_control']==temp_booster)][diff_col]+x_lag
        group1 = booster_first_orig[(booster_first_orig['diff_booster']>=0) & (booster_first_orig['case_control']==temp_booster)][real_col]
        group1_log = booster_first_orig[(booster_first_orig['diff_booster']>=0) & (booster_first_orig['case_control']==temp_booster)][log_col]
        diff_days_orig = booster_first_orig[(booster_first_orig['diff_booster']>=0) & (booster_first_orig['case_control']==temp_booster)][diff_col]

        vec_age = booster_first_orig[(booster_first_orig['diff_booster']>=0) & (booster_first_orig['case_control']==temp_booster)]['age_group']
        
        ## diff days
        if (temp_group=='first_val'):
            if (temp_booster==0) : vec_diff_control_vac2 = diff_days_orig
            else : vec_diff_case_vac2  = diff_days_orig
        else :
            if (temp_booster==0) : vec_diff_control_booster = diff_days_orig
            else : vec_diff_case_booster  = diff_days_orig
                
        ## IgG vals
        if (temp_group=='first_val'):
            if (temp_booster==0) : vec_vals_control_vac2 = group1
            else : vec_vals_case_vac2  = group1
        else :
            if (temp_booster==0) : vec_vals_control_booster = group1
            else : vec_vals_case_booster  = group1                
                
        
        #print('diff_days:',mean(diff_days_orig))
        x_val_mean = mean(diff_days)
        
        dots_x.extend(diff_days.tolist())
        dots_y.extend(group1.tolist())

        my_len = len(group1)
        log_mean1 = statistics.mean(group1_log)
        geom_mean1 = math.exp(log_mean1)
        se2 = sem(group1_log)
        geom_lower = math.exp(log_mean1-1.96*se2)
        geom_upper = math.exp(log_mean1+1.96*se2)
        
        if (temp_group=='first_val'): 
            vec_first=group1_log
            vec_first_orig=group1
        else : 
            vec_booster = group1_log   
            vec_booster_orig=group1
        
        print('case_control',temp_booster ,temp_group, my_len,geom_mean1,geom_lower,geom_upper )

        vals.append(geom_mean1)
        lower.append(geom_lower)
        upper.append(geom_upper)
        vec_x.append(x_val_mean)
    
        
    
    vec_fold = [m-n for m, n in zip(vec_booster, vec_first)]
    vec_fold_orig = [m/n for m, n in zip(vec_booster_orig, vec_first_orig)]
    
    import scipy.stats as st
    aa = st.t.interval(alpha=0.95, df=len(vec_fold)-1, loc=np.mean(vec_fold), scale=st.sem(vec_fold))  
    print('fold',np.exp(mean(vec_fold)), np.exp(aa[0]), np.exp(aa[1]))
    
    
    if (temp_booster==0) : 
        vec_fold_control = vec_fold
        vec_fold_control_orig = vec_fold_orig
        vec_age_control = vec_age
    else :  
        vec_fold_case = vec_fold
        vec_fold_case_orig = vec_fold_orig
        vec_age_case = vec_age
    
    
    #from scipy import stats
    #my_ttests = stats.ttest_ind(vec_first, vec_booster, equal_var=False)
    #print('ttest', my_ttests)
    print('mannwhitneyu',mannwhitneyu(vec_first, vec_booster))
    print ('=======================')
    
    ax_temp.plot(vec_x , vals, linewidth=my_width , label = my_label , linestyle=my_linestyle ,color=colors[0])
    ax_temp.scatter(vec_x ,vals ,  linewidth=my_width, facecolors =colors[0]  )
    
    
    diff1 = booster_first_orig[(booster_first_orig['diff_booster']>=0) & (booster_first_orig['case_control']==temp_booster)]['diff_between_val1_and_sec_vacc']+0
    val1 = booster_first_orig[(booster_first_orig['diff_booster']>=0) & (booster_first_orig['case_control']==temp_booster)]['first_val']

    diff2 = booster_first_orig[(booster_first_orig['diff_booster']>=0) & (booster_first_orig['case_control']==temp_booster)]['diff_booster']+65
    val2 = booster_first_orig[(booster_first_orig['diff_booster']>=0) & (booster_first_orig['case_control']==temp_booster)]['booster_val']
    
    
    ax_temp.plot([diff1,diff2] , [val1,val2], linewidth=0.20 , linestyle=my_linestyle ,color=colors[0])
    
    
    ax_temp.scatter(dots_x ,dots_y, s=5,c='#acc2d9',label=my_label_dots,alpha=0.4)
    
    for i,ind in enumerate(vec_x):
        ax_temp.plot([ind,ind],[lower[i], upper[i]], '_-' , markeredgewidth=3 , linewidth=3 , markersize=15 , color=colors[0])
            

            
            
    ax_temp.set_yscale('log')
    ax_temp.set_yticks([50,100,250,500,1000,2500,5000,10000,20000,40000])
    ax_temp.set_yticklabels([50,100,250,500,1000,2500,5000,10000,20000,40000],fontsize=14,fontname = "Times New Roman")
    ax_temp.set_ylim(40,None)
    if temp_booster==0:
        ax_temp.set_xticks(vec_x_ticks_0)
        ax_temp.set_xticklabels(vec_x_ticks_l_0,fontsize=14,fontname = "Times New Roman")
    else: 
        ax_temp.set_xticks(vec_x_ticks_1)
        ax_temp.set_xticklabels(vec_x_ticks_l_1,fontsize=14,fontname = "Times New Roman")
        
    #handles, labels = plt.gca().get_legend_handles_labels()
    #order = [0]
    #legend = ax_temp.legend([handles[idx] for idx in order],[labels[idx] for idx in order], shadow=True,prop={'family':'Times New Roman', 'size':13})

    if temp_booster==0:
        ax_temp.set_xlabel('      Days from Second Dose             Days from Booster                 ',fontsize=14,fontname = "Times New Roman")
    else:
        ax_temp.set_xlabel('      Days from Second Dose             Days from Booster                 ',fontsize=14,fontname = "Times New Roman")
        
    ax_temp.set_ylabel('IgG Values',fontsize=16,fontname = "Times New Roman")   
    ax_temp.legend(fontsize='large', loc='lower right')
    
    if temp_booster==0:
        ax_temp.set_title('immunocompetent    ',fontsize=16,fontname = "Times New Roman")
    else:
        ax_temp.set_title('Immunocompromised    ',fontsize=16,fontname = "Times New Roman")
        
    #ax_temp.axhline(y=20, color='black', linestyle='--',linewidth=1.3)
    ax_temp.axvline(x=62.5, color='black', linestyle='--',linewidth=1.3)
    
#plt.figtext(0.09,0.95,'A' , fontsize=18,weight='bold',fontname = "Times New Roman")
#plt.figtext(0.12,0.95,'Overall Population' , fontsize=18,fontname = "Times New Roman")  

#plt.figtext(0.37,0.98,'IgG compare Second Dose and Booster' , fontsize=18,fontname = "Times New Roman")  

plt.savefig('c:/temp/vaccine_antibody/compare_50_second_vs_booster_0112.jpg', dpi=300,bbox_inches = "tight")

plt.show()    

In [ ]:
print('vec_diff_control_vac2', mean(vec_diff_control_vac2), np.std(vec_diff_control_vac2))
print('vec_diff_control_booster', mean(vec_diff_control_booster),np.std(vec_diff_control_booster))
print('vec_diff_case_vac2' ,mean(vec_diff_case_vac2), np.std(vec_diff_case_vac2))
print('vec_diff_case_booster' ,mean(vec_diff_case_booster), np.std(vec_diff_case_booster))

In [ ]:
my_ttests = stats.ttest_ind(vec_diff_control_vac2, vec_diff_control_booster, equal_var=False)
print('control ttest', my_ttests)
my_ttests = stats.ttest_ind(vec_diff_case_vac2, vec_diff_case_booster, equal_var=False)
print('case ttest', my_ttests)

In [ ]:
vec_fold_case_orig_more_65 = vec_fold_case_orig
vec_fold_control_orig_more_65 = vec_fold_control_orig

In [ ]:
vec_fold_case_orig_less_65 = vec_fold_case_orig
vec_fold_control_orig_less_65 = vec_fold_control_orig


In [ ]:
print('mannwhitneyu',mannwhitneyu(vec_fold_control_orig_more_65, vec_fold_control_orig_less_65))

In [ ]:

#
import seaborn as sns, numpy as np
#ax = sns.boxplot(np.exp(vec_fold_case))

# vec_fold_case
# vec_fold_control

fig, ax = plt.subplots(1,1,figsize=(8,6))
aa = pd.DataFrame(zip(vec_fold_case_orig,vec_age_case), columns = ["fold","Age Group"])
aa['group'] = 'Immunocompromised'
bb = pd.DataFrame(zip(vec_fold_control_orig,vec_age_control), columns = ["fold","Age Group"])
bb['group'] = 'Immunocompetent'
cc=pd.concat([aa,bb])
#cc['my_exp'] = np.exp(cc['fold'])
cc['my_exp'] = cc['fold']
#sns.stripplot(x="my_exp", y="group", data=cc,size=4, color=".3", linewidth=0)
sns.boxplot(y="group", x="my_exp",data=cc,hue='Age Group')
ax.set_xlim(0,35)
#ax.set_xlabel('Group',fontsize=14)
ax.set_xlabel('IgG rate (level of Booster / level of second dose)',fontsize=14)
ax.set_ylabel('')
#ax.set_title("IgG Increate rate \n booster to second dose",fontsize=14)
ax.tick_params(axis='x', labelsize=14 )
ax.tick_params(axis='y', labelsize=14 )
plt.savefig('c:/temp/vaccine_antibody/fold_second_booster.jpg', dpi=300,bbox_inches = "tight")

In [ ]:
fold_df

In [ ]:
from scipy import stats

In [ ]:
vec_vals_control_vac2 = []
vec_vals_control_booster = []
vec_vals_case_vac2 = []
vec_vals_case_booster = []

In [ ]:
from scipy.stats import mannwhitneyu

In [ ]:
print('control-booster-val2',mannwhitneyu(vec_vals_control_vac2,vec_vals_control_booster ))
print('case-booster-val2',mannwhitneyu(vec_vals_case_vac2,vec_vals_case_booster))

print('case-control-val2',mannwhitneyu(vec_vals_case_vac2, vec_vals_control_vac2))
print('case-control-booster',mannwhitneyu(vec_vals_case_booster, vec_vals_control_booster))

In [ ]:
print('control-booster-val2',stats.ttest_ind(vec_diff_control_vac2,vec_diff_control_booster ))
print('case-booster-val2',stats.ttest_ind(vec_diff_case_vac2,vec_diff_case_booster), mean(vec_diff_case_vac2),mean(vec_diff_case_booster))


In [ ]:
k2, p = stats.normaltest(vec_diff_control_booster)
print(k2,p)

In [ ]:
from platform import python_version
python_version() 

In [ ]:
from scipy import stats
import math
r = stats.spearmanr(vec_vals_case_vac2, vec_vals_case_booster)[0]
print(stats.spearmanr(vec_vals_case_vac2, vec_vals_case_booster))
num = len(vec_vals_case_vac2)
stderr = 1.0 / math.sqrt(num - 3)
delta = 1.96 * stderr
lower = math.tanh(math.atanh(r) - delta)
upper = math.tanh(math.atanh(r) + delta)
print ("r %.6f,  lower %.6f upper %.6f  p %.6f " % (r , lower, upper , p))

In [ ]:
len(vec_vals_control_vac2)

In [ ]:
fig, ax_all = plt.subplots(1,2,figsize=(15,6))
colors = sns.color_palette(n_colors=5)

for immune in [0,1] : 
    
    ax = ax_all[immune]
    booster_compare = booster_first_orig[booster_first_orig['case_control']==immune]

    vacc2_first = booster_compare['first_val'].tolist()
    vacc3_first = booster_compare['booster_val'].tolist()

 
    
    #print('before', 'case_control', immune,'n', len(vacc2_last),'mean',vacc2_last.mean(), 'sd', statistics.stdev(vacc2_last) )
    #print('after', 'case_control', immune,'n', len(vacc3_first),'mean',vacc3_first.mean(), 'sd', statistics.stdev(vacc3_first) )
    
    
    points=list(set(zip(vacc2_first,vacc3_first))) 
    #print(len(points))
    #Generate a list of point counts
    count=[len([x for x,y in zip(vacc2_first,vacc3_first) if x==p[0] and y==p[1]]) for p in points]  
    #print(len(count))
    #print(len(vacc2_last))
    #print(len(vacc3_first))
    countx = [100*i**0.5 for i in count]
    #print(len(countx))
    ax.set_yscale('log')
    ax.set_xscale('log')
    ax.set_yticks([50,500,1000,2500,5000,10000,20000,40000,55000])
    ax.set_yticklabels([50,500,1000,2500,5000,10000,20000,40000,''],fontsize=12,fontname = "Times New Roman")

    ax.set_xticks([50,500,1000,2500,5000,10000,20000,40000,55000])
    ax.set_xticklabels([50,500,1000,2500,5000,10000,20000,40000,''],fontsize=12,fontname = "Times New Roman")

    ax.set_ylim(10,55000)
    ax.set_xlim(10,55000)
    ax.scatter( vacc2_first,vacc3_first  ,  linewidth=1,facecolors=colors[0],s=100, edgecolors='k', alpha=1  )

    ax.set_ylabel('IgG value after booster',fontsize=16,fontname = "Times New Roman")
    ax.set_xlabel('IgG value after Second Dose',fontsize=16,fontname = "Times New Roman")

    ax.plot([10, 45000], [10, 45000], ls="--", c=".3")
    
    if immune==0:
        ax.set_title('Immunocompetent',fontsize=16,fontname = "Times New Roman")
    else:
        ax.set_title('Immunocompromised',fontsize=16,fontname = "Times New Roman")

#fig.suptitle('IgG Compare after Second Dose and after Booster',fontsize=16,fontname = "Times New Roman", y=1.02)


plt.savefig('c:/temp/vaccine_antibody/igg_compare_second_dose_booster.jpg', dpi=600,bbox_inches = "tight")

plt.show()        


## Wanning

In [ ]:
anti_data_pre = pd.read_csv('c:/temp/vaccine_antibody/out_for_analysis_fixed_25112021.txt', delimiter='\t')

In [ ]:
anti_data_pre.shape

In [ ]:
anti_data.columns

In [ ]:
type(analysis_1_ids)

In [ ]:
anti_data =  anti_data_pre[anti_data_pre['id'].isin(analysis_1_ids)]

In [ ]:
anti_data.head()

In [ ]:
anti_data.shape

In [ ]:
#fig, ax = plt.subplots(1,1,figsize=(8,5),gridspec_kw={'width_ratios': [3, 2]})
fig, ax = plt.subplots(1,1,figsize=(8,5))
colors = sns.color_palette(n_colors=6)
my_width = 2.5
#https://stackoverflow.com/questions/22408237/named-colors-in-matplotlib
my_linestyle = 'solid'
gap = 0.1
my_label = 'Observed GM'
my_label_dots =  'Observed'
vec_x_ticks_0=  [20,30,60,90,120,150,180,210]
vec_x_ticks_l_0=  ['',30,60,90,120,150,180,210]

vec_x_ticks_1=  [20,30,60,90,120]
vec_x_ticks_l_1=  ['',30,60,90,120]




for temp_booster in [0] : 
    j = temp_booster

    ax_temp = ax#[temp_booster]
    my_mat_for_age = anti_data[(anti_data['booster_ind']==temp_booster)]
    
    for index , temp_age in enumerate(sorted(set(my_mat_for_age.immune_name))): 
        print('age', temp_age)
        
       
        if (temp_age=='Control'): my_label = 'Immunocompetent'
        else : my_label = 'Immunocompromised'

        
        my_mat = my_mat_for_age[(my_mat_for_age['immune_name']==temp_age)]
        print(my_mat.shape)
        dots_x = []
        dots_y = []
        vals = []
        lower = []
        upper = []
        vec_x = []    

        for temp_group in sorted(set(my_mat.group)):
            if (temp_group==8): continue


            diff_days = my_mat[(my_mat['group']==temp_group)]['diff_days']
            my_group1 = my_mat[(my_mat['group']==temp_group)]['group']
            group1 = my_mat[(my_mat['group']==temp_group)]['val']
            group1_log = my_mat[(my_mat['group']==temp_group)]['log_val']


            dots_x.extend(diff_days.tolist())
            dots_y.extend(group1.tolist())
            #ax_temp.scatter(temp_group ,group1, s=20,color='black')
            
            if (temp_group==7): continue
            if (temp_group==4): continue
            if (temp_group==5): continue                

            #group1_mean = geometric_mean(group1)
            my_len = len(group1)
            log_mean1 = statistics.mean(group1_log)
            geom_mean1 = math.exp(log_mean1)
            se2 = sem(group1_log)
            geom_lower = math.exp(log_mean1-1.96*se2)
            geom_upper = math.exp(log_mean1+1.96*se2)

            #print(temp_booster , temp_group,my_len,  geom_mean1,geom_lower,geom_upper)
            vals.append(geom_mean1)
            lower.append(geom_lower)
            upper.append(geom_upper)
            vec_x.append(temp_group*30)


        ax_temp.plot(vec_x , vals, linewidth=my_width , label = my_label , linestyle=my_linestyle ,color=colors[index])
        ax_temp.scatter(vec_x ,vals ,  linewidth=my_width, facecolors =colors[index]  )

        ax_temp.scatter(dots_x ,dots_y, s=10,color=colors[index],alpha=0.15, label='Observed')

        for i,ind in enumerate(vec_x):
            ax_temp.plot([ind,ind],[lower[i], upper[i]], '_-' , markeredgewidth=3 , linewidth=3 , markersize=15 , color=colors[index])


            
            
    ax_temp.set_yscale('log')
    ax_temp.set_yticks([50,100,250,500,1000,2500,5000,10000,20000,40000])
    ax_temp.set_yticklabels([50,100,250,500,1000,2500,5000,10000,20000,40000],fontsize=14,fontname = "Times New Roman")
    
    if temp_booster==0:
        ax_temp.set_xticks(vec_x_ticks_0)
        ax_temp.set_xticklabels(vec_x_ticks_l_0,fontsize=14,fontname = "Times New Roman")
    else: 
        ax_temp.set_xticks(vec_x_ticks_1)
        ax_temp.set_xticklabels(vec_x_ticks_l_1,fontsize=14,fontname = "Times New Roman")
        
    #handles, labels = plt.gca().get_legend_handles_labels()
    #order = [0]
    #legend = ax_temp.legend([handles[idx] for idx in order],[labels[idx] for idx in order], shadow=True,prop={'family':'Times New Roman', 'size':13})

    if temp_booster==0:
        ax_temp.set_xlabel('Days since Second Dose of Vaccine',fontsize=16,fontname = "Times New Roman")
    else:
        ax_temp.set_xlabel('Days since Booster Dose of Vaccine',fontsize=16,fontname = "Times New Roman")
        
    ax_temp.set_ylabel('IgG Values',fontsize=16,fontname = "Times New Roman")   
    ax_temp.legend(fontsize='medium', loc='upper right')
        
    ax_temp.axhline(y=50, color='black', linestyle='--',linewidth=1.3)
        
    #if temp_booster==0:
        #ax_temp.set_title('IgG values after Second Dose',fontsize=16,fontname = "Times New Roman")
    #else:
        #ax_temp.set_title('Booster',fontsize=16,fontname = "Times New Roman")            
        
#plt.figtext(0.09,0.97,'D' , fontsize=18,weight='bold',fontname = "Times New Roman")
#plt.figtext(0.12,0.97,'IgG by Immunosuppression Groups' , fontsize=18,fontname = "Times New Roman")
plt.savefig('c:/temp/vaccine_antibody/final_wanning_50_0112.jpg', dpi=600,bbox_inches = "tight")    

plt.show()



In [ ]:
len(anti_data.id.unique())

In [ ]:
anti_data['gender_male'] = 1-anti_data['gender_female']
anti_data['Time'] = anti_data['group']

In [ ]:
md = smf.mixedlm("log_val ~ Time +gender_male +age_higher_than_65+ imnnue_disease ", anti_data, groups=anti_data["id"], re_formula="~Time")
mdf = md.fit(method=["powell"],maxiter=10)
print(mdf.summary())

## before and after booster

In [ ]:
booster_last_orig_pre = pd.read_csv('c:/temp/vaccine_antibody/out_for_analysis_compare_last_25112021.txt', delimiter='\t')

In [ ]:
#booster_last_orig_pre.to_csv('c:/temp/vaccine_antibody/out_for_analysis_compare_last_10112021_orig.txt', sep='\t', index=None)

In [ ]:
booster_last_orig_pre.shape

In [ ]:
booster_last_orig = booster_last_orig_pre[booster_last_orig_pre['val1']>=50]

In [ ]:
booster_last_orig.shape

In [ ]:
booster_last_orig.head()

In [ ]:
booster_last_orig[booster_last_orig['id']==420]

In [ ]:
booster_last_orig.case_control.value_counts()

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(14,5),gridspec_kw={'width_ratios': [1,1]})
colors = sns.color_palette(n_colors=6)
my_width = 3

my_linestyle = 'solid'
gap = 0.1
my_label = 'Observed GM'
my_label_dots =  'Observed'
#vec_x_ticks_0=  [-60,-30,0,30,60]
#vec_x_ticks_l_0=  [-60,-30,0,30,60]

#vec_x_ticks_1=  [-60,-30,0,30,60]
#vec_x_ticks_l_1=  [-60,-30,0,30,60]

# vec_x_ticks=  [20,30,60,90]
# vec_x_ticks_l=  ['',30,60,90]



vec_diff_control_vac2 = []
vec_diff_control_booster = []
vec_diff_case_vac2 = []
vec_diff_case_booster = []



vec_val_control_vac2 = []
vec_val_control_booster = []
vec_val_case_vac2 = []
vec_val_case_booster = []


vec_fold_control_orig = []
vec_fold_case_orig = []

vec_age_control =[]
vec_age_case =[]

for temp_booster in [0,1] : 
    
    if (temp_booster==0) : print('@@@@@  control @@@@@@')
    else :  print('@@@@@  case @@@@@@')    
    
    
    #if (temp_booster==0): my_label = 'Immunocompetent'
    #else : my_label = 'Immunocompromised'    
    
    j = temp_booster
    vals = []
    lower = []
    upper = []
    vec_x = []
    ax_temp = ax[temp_booster]
    
    dots_x = []
    dots_y = []
    
    vec_last = []
    vec_booster = []    
    
    vec_last_orig = []
    vec_booster_orig = []        

    for temp_group in ['val_last','booster_val']:
            
        if (temp_group=='val_last'):
            log_col = 'log_last'
            real_col = 'val_last'
            diff_col = 'diff_val_before_to_booster'
            diff_col2 = 'diff_val_before_to_booster' #'last_val_to_vacc2'
            #x_val =1
            x_lag = 0
        else:
            log_col = 'log_booster'
            real_col = 'booster_val'
            diff_col = 'diff_val_booster_to_booster'
            diff_col2 = 'diff_val_booster_to_booster'
            #x_val = 3
            x_lag = 0

        diff_days_orig = booster_last_orig[(booster_last_orig['case_control']==temp_booster)][diff_col2]
        diff_days = booster_last_orig[(booster_last_orig['case_control']==temp_booster)][diff_col]+x_lag
        group1 = booster_last_orig[ (booster_last_orig['case_control']==temp_booster)][real_col]
        group1_log = booster_last_orig[ (booster_last_orig['case_control']==temp_booster)][log_col]
        vec_age = booster_last_orig[(booster_last_orig['case_control']==temp_booster)]['age_group']
        
        ## diff days
        if (temp_group=='val_last'):
            if (temp_booster==0) : vec_diff_control_vac2 = diff_days_orig
            else : vec_diff_case_vac2  = diff_days_orig
        else :
            if (temp_booster==0) : vec_diff_control_booster = diff_days_orig
            else : vec_diff_case_booster  = diff_days_orig        
        
        
        
        ## diff days
        if (temp_group=='val_last'):
            if (temp_booster==0) : vec_val_control_vac2 = group1
            else : vec_val_case_vac2  = group1
        else :
            if (temp_booster==0) : vec_val_control_booster = group1
            else : vec_val_case_booster  = group1          
        
        
        #print(mean(diff_days))
        x_val = mean(diff_days)
        
        dots_x.extend(diff_days.tolist())
        dots_y.extend(group1.tolist())

        my_len = len(group1)
        log_mean1 = statistics.mean(group1_log)
        geom_mean1 = math.exp(log_mean1)
        se2 = sem(group1_log)
        geom_lower = math.exp(log_mean1-1.96*se2)
        geom_upper = math.exp(log_mean1+1.96*se2)

        vals.append(geom_mean1)
        lower.append(geom_lower)
        upper.append(geom_upper)
        vec_x.append(x_val)
        
        
        print('case_control',temp_booster ,temp_group, my_len,geom_mean1,geom_lower,geom_upper )
        
        if (temp_group=='val_last'): 
            vec_last=group1_log
            vec_last_orig = group1
        else : 
            vec_booster = group1_log  
            vec_booster_orig = group1
    
    
    print('n=', len(vec_last), len(vec_booster))
    
    vec_fold = [m-n for m, n in zip(vec_booster, vec_last)]
    import scipy.stats as st
    aa = st.t.interval(alpha=0.95, df=len(vec_fold)-1, loc=np.mean(vec_fold), scale=st.sem(vec_fold))  
    print('fold',np.exp(mean(vec_fold)), np.exp(aa[0]), np.exp(aa[1]))
    
    vec_fold_orig = [m/n for m, n in zip(vec_booster_orig, vec_last_orig)]
    
    if (temp_booster==0) : 
        vec_fold_control_orig = vec_fold_orig
    else :  
        vec_fold_case_orig = vec_fold_orig
    
    if (temp_booster==0) : 
        vec_age_control = vec_age
    else :  
        vec_age_case = vec_age    
    
    
    
    from scipy import stats
    my_ttests = mannwhitneyu(vec_last, vec_booster)
    print('ttest', my_ttests)
    print ('=======================')    

    ax_temp.plot(vec_x , vals, linewidth=my_width , label = my_label , linestyle=my_linestyle ,color=colors[0])
    ax_temp.scatter(vec_x ,vals ,  linewidth=my_width, facecolors =colors[0]  )
    
    
    diff1 = booster_last_orig[(booster_last_orig['case_control']==temp_booster)]['diff_val_before_to_booster']
    val1 = booster_last_orig[(booster_last_orig['case_control']==temp_booster)]['val_last']

    diff2 = booster_last_orig[(booster_last_orig['case_control']==temp_booster)]['diff_val_booster_to_booster']
    val2 = booster_last_orig[(booster_last_orig['case_control']==temp_booster)]['booster_val']
    
    
    ax_temp.plot([diff1,diff2] , [val1,val2], linewidth=0.1 , linestyle=my_linestyle ,color=colors[0])
    
    
    ax_temp.scatter(dots_x ,dots_y, s=10,c='#acc2d9',label=my_label_dots,alpha=0.4)
    
    for i,ind in enumerate(vec_x):
        ax_temp.plot([ind,ind],[lower[i], upper[i]], '_-' , markeredgewidth=3 , linewidth=3 , markersize=15 , color=colors[0])
            

            
            
    ax_temp.set_yscale('log')
    ax_temp.set_yticks([20,50,100,250,500,1000,2500,5000,10000,20000,40000])
    ax_temp.set_yticklabels(['',50,100,250,500,1000,2500,5000,10000,20000,40000],fontsize=14,fontname = "Times New Roman")
    ax_temp.set_xlim(-80,80)
    
    #if temp_booster==0:
        #ax_temp.set_xticks(vec_x_ticks_0)
        #ax_temp.set_xticklabels(vec_x_ticks_l_0,fontsize=14,fontname = "Times New Roman")
    #else: 
        #ax_temp.set_xticks(vec_x_ticks_1)
        #ax_temp.set_xticklabels(vec_x_ticks_l_1,fontsize=14,fontname = "Times New Roman")
        
    #handles, labels = plt.gca().get_legend_handles_labels()
    #order = [0]
    #legend = ax_temp.legend([handles[idx] for idx in order],[labels[idx] for idx in order], shadow=True,prop={'family':'Times New Roman', 'size':13})

    if temp_booster==0:
        ax_temp.set_xlabel('Days from Booster',fontsize=16,fontname = "Times New Roman")
    else:
        ax_temp.set_xlabel('Days from Booster',fontsize=16,fontname = "Times New Roman")
        
    ax_temp.set_ylabel('IgG Values',fontsize=16,fontname = "Times New Roman")   
    ax_temp.legend(fontsize='large', loc='lower right')
    
    if temp_booster==0:
        ax_temp.set_title('Immunocompetent',fontsize=16,fontname = "Times New Roman")
    else:
        ax_temp.set_title('Immunocompromised',fontsize=16,fontname = "Times New Roman")
        
    #ax_temp.axhline(y=20, color='black', linestyle='--',linewidth=1.3)
    ax_temp.axvline(x=0, color='black', linestyle='--',linewidth=1.3)
    
#plt.figtext(0.36,1.00,'IgG compare before and after the Booster' , fontsize=18,fontname = "Times New Roman")  
    
plt.savefig('c:/temp/vaccine_antibody/compare_50_before_after_booster_0112.jpg', dpi=600,bbox_inches = "tight")

plt.show()    

In [ ]:
print('control days', stats.ttest_ind(vec_diff_control_vac2, vec_diff_control_booster, equal_var=False))
print('case days', stats.ttest_ind(vec_diff_case_vac2, vec_diff_case_booster, equal_var=False))

In [ ]:

vec_diff_control_vac2 = []
vec_diff_control_booster = []
vec_diff_case_vac2 = []
vec_diff_case_booster = []


In [ ]:
print('vec_diff_control_vac2', mean(vec_diff_control_vac2), np.std(vec_diff_control_vac2))
print('vec_diff_control_booster', mean(vec_diff_control_booster),np.std(vec_diff_control_booster))
print('vec_diff_case_vac2' ,mean(vec_diff_case_vac2), np.std(vec_diff_case_vac2))
print('vec_diff_case_booster' ,mean(vec_diff_case_booster), np.std(vec_diff_case_booster))

In [ ]:
print('before booster: compare case control', mannwhitneyu(vec_val_control_vac2, vec_val_case_vac2))
print('after booster: compare case control', mannwhitneyu(vec_val_control_booster, vec_val_case_booster))

In [ ]:
vec_val_control_vac2 = []
vec_val_control_booster = []
vec_val_case_vac2 = []
vec_val_case_booster = []

In [ ]:



#
import seaborn as sns, numpy as np
#ax = sns.boxplot(np.exp(vec_fold_case))

# vec_fold_case
# vec_fold_control

fig, ax = plt.subplots(1,1,figsize=(8,6))
aa = pd.DataFrame(zip(vec_fold_case_orig,vec_age_case), columns = ["fold","Age Group"])
aa['group'] = 'Immunocompromised'
bb = pd.DataFrame(zip(vec_fold_control_orig,vec_age_control), columns = ["fold","Age Group"])
bb['group'] = 'Immunocompetent'
cc=pd.concat([aa,bb])
#cc['my_exp'] = np.exp(cc['fold'])
cc['my_exp'] = cc['fold']
ax = sns.boxplot(y="group", x="my_exp",data=cc,hue='Age Group')
ax.set_xlim(0,200)
#ax.set_xlabel('Group',fontsize=14)
ax.set_xlabel('IgG rate (level after Booster / level before booster)',fontsize=14)
#ax.set_xlabel('Increase rate',fontsize=14)
ax.set_ylabel('')
#ax.set_title("IgG Increate rate \n before and after the booster",fontsize=14)
ax.tick_params(axis='x', labelsize=14 )
ax.tick_params(axis='y', labelsize=14 )
plt.savefig('c:/temp/vaccine_antibody/fold_increase_before_after_0112.jpg', dpi=300,bbox_inches = "tight")

In [ ]:
# diff prior to booster

diff_before_case = booster_last_orig[booster_last_orig['case_control']==1]['diff_val_before_to_booster']
diff_before_control = booster_last_orig[booster_last_orig['case_control']==0]['diff_val_before_to_booster']


print('before booster - case',len(diff_before_case), mean(diff_before_case), np.std(diff_before_case))
print('before booster - control',len(diff_before_control), mean(diff_before_control),np.std(diff_before_control))

## neut

In [ ]:
neut_mat = pd.read_csv('c:/temp/vaccine_antibody/naturelizing_03_12_2021.txt', delimiter='\t')

In [ ]:
neut_mat.head()

In [ ]:
neut_mat['date_new'] = pd.to_datetime(neut_mat["date"])

In [ ]:
neut_mat.dtypes

In [ ]:
print(neut_mat.shape)
#neut_mat = neut_mat[(neut_mat['date_new'] > '2021-05-01')]
print(neut_mat.shape)

In [ ]:
neut_mat.shape

In [ ]:
neut_mat.dtypes

In [ ]:
mat_in_cohort = pd.read_csv('c:/temp/vaccine_antibody/id_in_cohort.txt', delimiter='\t')
my_id_list = mat_in_cohort[mat_in_cohort['in_analysis']==1]['id'].to_list()
len(my_id_list)

In [ ]:
neut_mat_in_cohort = neut_mat[neut_mat['id'].isin (my_id_list)]
print('neut_mat',neut_mat.shape)
print('neut_mat_in_cohort',neut_mat_in_cohort.shape)

In [ ]:
from scipy import stats
import math
r = stats.spearmanr(neut_mat_in_cohort['val'], neut_mat_in_cohort['val_neut'])[0]
print(stats.spearmanr(neut_mat_in_cohort['val'], neut_mat_in_cohort['val_neut']))
num = len(neut_mat_in_cohort['val'])
stderr = 1.0 / math.sqrt(num - 3)
delta = 1.96 * stderr
lower = math.tanh(math.atanh(r) - delta)
upper = math.tanh(math.atanh(r) + delta)
print ("r %.6f,  lower %.6f upper %.6f  p %.6f " % (r , lower, upper , p))

In [ ]:
from matplotlib import pyplot

fig, ax_temp = plt.subplots(1,1,figsize=(7,6))
ax_temp.set_yscale('log')
ax_temp.set_yticks([20,50,100,250,500,1000,2500,5000,10000,20000,40000])
ax_temp.set_yticklabels(['',50,100,250,500,1000,2500,5000,10000,20000,40000],fontsize=14,fontname = "Times New Roman")

ax_temp.set_xscale('log')
ax_temp.set_xticks([20,50,100,250,500,1000,2500,5000,10000,20000,40000])
ax_temp.set_xticklabels(['',50,100,250,500,1000,2500,5000,10000,20000,40000],fontsize=14,fontname = "Times New Roman")

ax_temp.set_xlim(50,40000)
ax_temp.set_ylim(50,10000)

ax_temp.scatter(neut_mat_in_cohort['val'], neut_mat_in_cohort['val_neut'],alpha=0.6,linewidths=2,s=100)
ax_temp.set_xlabel("IgG values",fontsize=14)
ax_temp.set_ylabel("Neutrelizing values",fontsize=14)

ax_temp.text(100, 5000, 'Spearman correlation = 0.77 [0.67-0.85] \np-value < 0.0001', color='red',fontsize=12)

name = 'c:/temp/vaccine_antibody/neut_100_pop_0112.jpg'
plt.savefig(name, dpi=300,bbox_inches = "tight")
plt.show()


In [ ]:
len(neut_mat_in_cohort['val'])

In [ ]:
len(neut_mat_in_cohort.id.unique())

In [ ]:
neut_mat.head()

In [ ]:
print(neut_mat.shape)
print(mat_in_cohort.shape)
neut_mat2 = pd.merge(neut_mat,mat_in_cohort[mat_in_cohort['in_analysis']==1], on='id', how='inner' )
print(neut_mat2.shape)

In [ ]:
neut_mat2.head()

In [ ]:

for i in (0,1,2):
    #print(i)
    if (i==0):
        vec1= neut_mat2[neut_mat2['age_group']=='<65 yr']['val']
        vec2=  neut_mat2[neut_mat2['age_group']=='<65 yr']['val_neut']
        n_id = len(neut_mat2[neut_mat2['age_group']=='<65 yr']['id'].unique())
        name = 'less than 65'
    elif (i==1):
        vec1= neut_mat2[neut_mat2['age_group']=='>=65 yr']['val']
        vec2=  neut_mat2[neut_mat2['age_group']=='>=65 yr']['val_neut']
        name = 'more 65 yr'     
        n_id = len(neut_mat2[neut_mat2['age_group']=='>=65 yr']['id'].unique())
    elif (i==2):
        vec1= neut_mat2['val']
        vec2=  neut_mat2['val_neut']
        name = 'all'       
        n_id = len(neut_mat2['id'].unique())

    from scipy import stats
    import math
    r = stats.spearmanr(vec1, vec2)[0]
    #print(stats.spearmanr(vec1, vec2))
    num = len(vec1)
    stderr = 1.0 / math.sqrt(num - 3)
    delta = 1.96 * stderr
    lower = math.tanh(math.atanh(r) - delta)
    upper = math.tanh(math.atanh(r) + delta)
    print (name ,len(vec1),'n-id',n_id ,  "r %.6f,  lower %.6f upper %.6f  p %.6f " % (r , lower, upper , p))